In [1]:
import pandas as pd
import re
import operator
import calendar
import urllib.request, urllib.error, urllib.parse
from datetime import datetime
from bs4 import BeautifulSoup
# Get Tourneys
fileName=('BUWomensRecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    tourneyDict={}
    for i in rows:
        row=i.split('  ')
        if(len(row)==2):
            tourneyDict[row[0]]=row[1]
            #print(row)
            

In [84]:
# Get Games
fileName=('BUWomensRecordBook.txt')
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:      
        if(len(i)==7):
            season=i 
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN.?: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w) (?:\((.?ot)\))? ?(.*) (\w) (#\d*)? ?(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(7),
                 'gameType':gameType,
                 'tourney': game.group(8),
                 'scoreline':game.group(9),
                 'location':game.group(5),
                 'rank':game.group(6),
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['tourney']=='(ex)' or gameDict['result']=='E'):
            gameDict['gameType']='Exhibition'
            gameDict['result'] = 'E'
            gameDict['tourney'] = None

        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        if((gameDict['tourney'] == gameDict['conference'] +" "+ 'Tournament') or (gameDict['tourney'] == 'NCAA Tournament')):
            gameDict['seasonType'] = 'Playoffs'
        else:
            gameDict['seasonType'] = 'Regular Season'
        if(gameDict['location']=='H'):
            gameDict['location']='Home'
        elif(gameDict['location']=='A'):
            gameDict['location']='Away'
        elif(gameDict['location']=='N'):
            gameDict['location']='Neutral'
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        gameDict['month']=int(gameDict['month'])
        gameDict['day']=int(gameDict['day'])
        if(gameDict['month'] >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=int(gameDict['season'][:4])
        elif(gameDict['month'] <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        gameDict['date']=pd.Timestamp(gameDict['date'])
        gameDict['dow']=gameDict['date'].weekday()
        gameList.append(gameDict)
f.close()
dfWomensGames=pd.DataFrame(gameList)

In [4]:
import urllib.request, urllib.error, urllib.parse
from bs4 import BeautifulSoup
import re

skaterSeasonList=[]
goalieSeasonList=[]
#seasons=["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]
seasons=["2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]

for season in seasons:
    sSeason=season[2:4]+season[-2:]
    url = "http://www.collegehockeystats.net/{}/teamstats/bu_w".format(sSeason)
    #print(url)
    f=urllib.request.urlopen(url)
    html = f.read()
    f.close()
    soup = BeautifulSoup(html, 'html.parser')
    tab = soup.find_all("table")
    skatersTab=tab[2]
    tabRows=skatersTab.find_all('tr')
    statsDict={"number" : 0,
    "name" : 1,
    "pos" : 2,
    "yr" : 3,
    "overall_gp" : 4,
    "overall_goals" : 5,
    "overall_assists" : 6,
    "overall_pts" : 7,
    "overall_pen" : 8,
    "overall_ppg" : 9,
    "overall_shg" : 10,
    "overall_gwg" : 11,
    "overall_plusminus" : 12,
    "overall_sog" : 13,
    "conf_gp" : 14,
    "conf_goals" : 15,
    "conf_assists" : 16,
    "conf_pts" : 17,
    "conf_pen" : 18,
    "conf_ppg" : 19,
    "conf_shg" : 20,
    "conf_gwg" : 21,
    "conf_plusminus" : 22,
    "conf_sog" : 23,
    "career_gp" : 24,
    "career_goals" : 25,
    "career_assists" : 26,
    "career_pts" : 27}

    gStatsDict={"number" : 0,
    "name" : 1,
    "yr" : 2,
    "gp" : 3,
    "mins" : 4,
    "ga" : 5,
    "saves" : 6,
    "shots" : 7,
    "saveperc" : 8,
    "gaa" : 9,
    "record" : 10,
    "winpec" : 11,
    "gs" : 12,
    "so" : 13}

    skateRows=skatersTab.find_all('tr', class_=lambda x: x != 'chssmallbold')
    for i in range(1,len(skateRows)-1):
        playerDict={}
        for d in statsDict.keys():
                playerDict[d] = skateRows[i].find_all('td')[statsDict[d]].get_text().lstrip(' ').rstrip(' ').replace('\xa0','N/A')
        playerDict['name']=re.sub(' \(.*','',playerDict['name'])
        playerDict['season']=season

        skaterSeasonList.append(playerDict)


    goalieTab=tab[3]
    goalieRows=goalieTab.find_all('tr', class_=lambda x: x != 'chssmallbold')

    for i in range(1,len(goalieRows)):
        playerDict={}
        if("Open Net" in goalieRows[i].get_text() or "##" in goalieRows[i].get_text()):
            break
        for d in gStatsDict.keys():
            playerDict[d] = goalieRows[i].find_all('td')[gStatsDict[d]].get_text().lstrip(' ').rstrip(' ').replace('\xa0','N/A')
        playerDict['name']=re.sub(' \(.*','',playerDict['name'])
        playerDict['season']=season
        goalieSeasonList.append(playerDict)



In [10]:
dfSkateNew=pd.DataFrame(skaterSeasonList)

In [33]:
dfGoalNew=pd.DataFrame(goalieSeasonList)

In [12]:
seasons=["2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14"]
#seasons=["2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]
playerList=[]
goalieList=[]
for season in seasons:
    sSeason=season[2:4]+season[-2:]
    url = "http://www.collegehockeystats.net/{}/teamstats/bu_w".format(sSeason)
    #print(url)
    f=urllib.request.urlopen(url)
    html = f.read()
    f.close()
    soup = BeautifulSoup(html, 'html.parser')
    table= soup.find("pre")
    rows=table.get_text().split('\n')
    goalies=False
    for i in rows:
        if('Goaltending (Overall)' in i):
            goalies=True
        if('Goaltending (Conference Only)' in i):
            break
        if('--' not in i and 'Boston University' not in i and 'Opponents' not in i and '##' not in i and 'Bench' not in i and not goalies):
            name,overall,conf,career=i.split('|')
            name=re.sub('\(.*\)','',name)
            name=name.split(' ')
            overall=overall.split(' ')
            name=list(filter(None,name))
            overall=list(filter(None,overall))
            if(overall[4]=='0'):
                overall[4]+='/'
            playDict={'number':name[0],
                      'name':name[1]+ " " +name[2],
                      'pos':name[3],
                      'yr':name[4],
                      'gp':overall[0],
                      'goals':overall[1],
                      'assists':overall[2],
                      'pts':overall[3],
                      'pen':overall[4]+overall[5],
                      'season':season}
            playerList.append(playDict)
        if('----' not in i and 'Boston University' not in i and 'Opponents' not in i and '##' not in i and 'Open' not in i and goalies):
            name,overall,none=i.split('|')
            name=re.sub('\(.*\)','',name)
            name=name.split(' ')
            overall=overall.split(' ')
            name=list(filter(None,name))
            overall=list(filter(None,overall))
            if(len(overall)==13):
                goalieDict={'number':name[0],
                          'name':name[1]+ " " +name[2],
                          'yr':name[3],
                          'gp':overall[0],
                          'mins':overall[1],
                          'ga':overall[2],
                          'saves':overall[3],
                          'sv%':overall[5],
                          'gaa':overall[6],
                          'record':overall[7]+overall[8],
                          'SO':overall[11],
                          'season':season}
            else:
                goalieDict={'number':name[0],
                      'name':name[1]+ " " +name[2],
                      'yr':name[3],
                      'gp':overall[0],
                      'mins':overall[1],
                      'ga':overall[2],
                      'saves':overall[3],
                      'sv%':overall[5],
                      'gaa':overall[6],
                      'record':overall[7]+overall[8]+overall[9],
                      'SO':overall[11],
                      'season':season}
            goalieList.append(goalieDict)
dfPlay=pd.DataFrame(playerList)
dfGoal=pd.DataFrame(goalieList)

In [27]:
for i in dfPlay.to_string(index=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

number	name	pos	yr	gp goals assists pts	pen	 season
17	 Laurel Koller	F	FR	33	13	15	28	17/34	2005-06
 9	 Gina Kearns	F	FR	33	13	14	27	 7/14	2005-06
22	 Nicki Wiart	F	FR	33	12	13	25	 7/14	2005-06
 8	 Cara Hendry	F	JR	33	 7	14	21	31/62	2005-06
10	Erin Seman	F	FR	31	 7	11	18	33/96	2005-06
 4	 Amanda Shaw	D	FR	33	 3	 9	12	32/64	2005-06
21	 Stephanie Armstrong	F	SO	33	 4	 5	 9	2/4	2005-06
19	 Genevieve Laing	F	FR	33	 6	 2	 8	2/4	2005-06
14	 Caroline Bourdeau	F	FR	33	 4	 4	 8	 7/14	2005-06
12	 Kelley Karnan	F	FR	32	 4	 1	 5	4/8	2005-06
 6	Jessica Lortie	F	JR	32	 1	 4	 5	12/32	2005-06
 2	 Sarah Russell	D	FR	33	 1	 4	 5	16/32	2005-06
 7	Maeghan Ouimet	F	FR	31	 1	 3	 4	10/20	2005-06
 5	 Caitlin Reddy	D	FR	33	 0	 4	 4	10/20	2005-06
13	 Catherine Foley	D	FR	32	 1	 2	 3	 6/12	2005-06
11	 Emily Schwarz	D	FR	33	 0	 2	 2	18/36	2005-06
28	 Sophie Thornton	D	FR	33	 1	 0	 1	3/6	2005-06
 3	 Erica Christensen	F	FR	14	 0	 1	 1	1/2	2005-06
29	Justine McLaughlin	D	FR	 1	 0	 0	 0	0/0	2005-06
 1	Eden Spencer	G

In [17]:
dfSkate=dfSkateNew[['number','name','pos','yr','overall_gp','overall_goals','overall_assists','overall_pts','overall_pen','season']]

In [29]:
for i in dfGoal.to_string(index=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

number	 name	yr	gp	 mins	ga saves	sv%	 gaa	 record	SO	 season
39	Allyse Wilcox	FR	28	1597:19	70	 736	 .913	2.63	 9-14-4	 1	2005-06
23	Karilyn Pilch	FR	 8	 395:52	24	 184	 .885	3.64	3-3-0	 6	2005-06
 1	 Eden Spencer	FR	 1	 3:03	 0	 2	1.000	0.00	0-0-0	 0	2005-06
39	Allyse Wilcox	SO	20	1168:48	48	 520	 .915	2.46	 10-9-1	20	2006-07
33	Melissa Haber	FR	15	 879:25	31	 312	 .910	2.12	9-3-2	14	2006-07
39	Allyse Wilcox	JR	26	1534:16	50	 613	 .925	1.96	12-12-2	 3	2007-08
33	Melissa Haber	SO	10	 551:21	36	 271	 .883	3.92	3-5-1	 9	2007-08
 1	Ashley Leichliter	SO	 2	31:29	 0	 5	1.000	0.00	0-0-0	 0	2007-08
39	Allyse Wilcox	SR	24	1436:15	54	 549	 .910	2.26	 12-8-4	24	2008-09
33	Melissa Haber	JR	12	 732:02	20	 258	 .928	1.64	6-3-3	12	2008-09
 1	Ashley Leichliter	JR	 2	15:09	 2	 4	 .667	7.92	0-0-0	 0	2008-09
33	Melissa Haber	SR	32	1906:23	65	 773	 .922	2.05	 15-8-8	32	2009-10
30	 Alissa Fromkin	FR	 8	 439:17	19	 143	 .883	2.60	2-1-4	 6	2009-10
 1	Kerrin Sperry	FR	28	1689:52	46	 601	 .929	1.63	 21-4-3	2

In [36]:
dfGN=dfGoalNew[['number','name','yr','gp','mins','ga','saves','saveperc','gaa','record','so','season']]

In [37]:
for i in dfGN.to_string(index=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

number	 name	yr	gp	 mins	ga saves saveperc	 gaa	record so	 season
33	Victoria Hanson	SO	23	1332:17	50	 506	 .910	2.25	16-5-2	0	2014-15
31	Erin O'Neil	FR	16	 870:00	39	 362	 .903	2.69	 9-4-1	0	2014-15
30	 Mia Becker	SO	 2	23:04	 2	12	 .857	5.20	 0-0-0	0	2014-15
31	Erin O'Neil	SO	30	1671:13	81	 809	 .909	2.91	17-8-2	2	2015-16
33	Victoria Hanson	JR	12	 655:53	35	 278	 .888	3.20	 6-6-0	0	2015-16
 1	Ashlyn Aiello	SO	 2	11:36	 0	 6	1.000	0.00	 0-0-0	0	2015-16
33	Victoria Hanson	SR	28	1693:17	67	 845	 .927	2.37	15-9-4	2	2016-17
31	Erin O'Neil	JR	 9	 547:12	34	 275	 .890	3.73	 4-3-2	0	2016-17
30	Corinne Schroeder	FR	25	1436:17	60	 633	 .913	2.51	12-8-4	0	2017-18
31	Erin O'Neil	SR	14	 805:50	35	 412	 .922	2.61	 2-9-2	1	2017-18
 1	Ashlyn Aiello	SR	 1	 6:57	 0	 5	1.000	0.00	 0-0-0	0	2017-18
30	Corinne Schroeder	SO	32	1930:17	61	 846	 .933	1.90	18-7-7	2	2018-19
 1	Kate Stuart	FR	 5	 303:38	 9	 149	 .943	1.78	 3-1-1	0	2018-19
31	Nicole McGuigan	FR	 2	29:11	 0	 3	1.000	0.00	 0-0-0	0	2018-19
30	Corin

In [31]:
url = "https://goterriers.com/sports/womens-ice-hockey/stats/?path=whockey"
#print(url)
f=urllib.request.urlopen(url)
html = f.read()
f.close()
soup = BeautifulSoup(html, 'html.parser')
curSkate=soup.find('section',{'id':'individual-overall-skaters'})
rows=curSkate.find_all('tr')
currSkaters=[]
for i in rows:
    col=i.find_all('td')
    name=i.find('span')
    if(name!=None):
        counter=0
        if('Team' in name):
            break
        lastName,firstName=name.get_text().split(', ')
        skateDict={'number':int(col[0].get_text()),
                   'last':lastName,
                   'first':firstName,
                   'name':firstName+' '+lastName,
                    'gp':int(col[1].get_text()),
                    'goals':int(col[2].get_text()),
                    'assists':int(col[3].get_text()),
                    'pts':int(col[4].get_text()),
                    'pen':col[16].get_text().replace('-','/')}
        currSkaters.append(skateDict)
dfCurr=pd.DataFrame(currSkaters)


In [32]:
dfC= dfCurr[['number','name','gp','goals','assists','pts','pen']]
for i in dfC.to_string(index=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

 number	name	gp	goals	assists	pts	pen
5	Courtney Correia	29	 13	9	 22	4/8
 16	Julia Nearis	28	4	 16	 20	 7/14
 11	 Mackenna Parker	30	9	7	 16	10/20
 17	Lacey Martin	28	5	7	 12	3/6
 28	Catherine Foulem	31	4	8	 12	4/8
6	 Nadia Mattivi	27	3	9	 12	14/28
 20	 Julia Shaunessy	26	4	6	 10	16/32
 19	 Emma Wuthrich	31	5	4	9	11/22
 18	Haylee Blinkhorn	30	3	5	8	1/2
 22	Christina Vote	31	5	2	7	 7/14
2	Alex Allan	31	3	4	7	 6/12
 15	Ellie Larson	24	2	5	7	0/0
7	 Kylie Roberts	14	4	2	6	1/2
 13	Andi Calderone	31	0	6	6	4/8
9	 Clare O'Leary	24	1	4	5	1/2
 10	Jesse DeVito	27	0	5	5	 8/16
 23	Tamara Giaquinto	25	0	5	5	1/2
 12	Kaleigh Donnelly	26	1	3	4	1/2
5	Courtney Correia	 2	1	2	3	0/0
 21	 Madison Cardaci	17	0	3	3	4/8
 11	 Mackenna Parker	 2	1	1	2	1/2
 26	Brooke Ersoy	17	0	2	2	3/6
 20	 Julia Shaunessy	 2	0	2	2	1/2
 16	Julia Nearis	 2	1	0	1	0/0
 19	 Emma Wuthrich	 2	1	0	1	2/4
 21	 Madison Cardaci	 2	0	1	1	0/0
 23	Tamara Giaquinto	 1	0	1	1	0/0
2	Alex Allan	 2	0	0	0	0/0
 18	Haylee Blinkhorn	 2	0	0	0	0/0
 13	An

In [64]:
fileName=('SeasonSkaterStatsWomens.txt')
jerseyList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    for i in rows:
        col=i.split('\t')
        jerseyDict={'number':int(col[0]),
                    'player':col[1],
                    'season':col[-1]}
        jerseyList.append(jerseyDict)           

In [101]:
dfJersey=pd.DataFrame(jerseyList)

In [102]:
dfJersey['season']=dfJersey.groupby(['number','player'])['season'].transform(lambda x: ','.join(x))
dfJersey[['number','player','season']].drop_duplicates()

,number,player,season
0,17,Laurel Koller,"2005-06,2007-08,2008-09,2009-10"
1,9,Gina Kearns,"2005-06,2006-07,2007-08,2008-09"
2,22,Nicki Wiart,"2005-06,2006-07,2007-08,2008-09"
3,8,Cara Hendry,"2005-06,2006-07"
4,10,Erin Seman,"2005-06,2006-07,2007-08,2008-09"
...,...,...,...
385,10,Jesse DeVito,2021-22
389,14,Madison Michals,2021-22
390,15,Ellie Larson,2021-22
397,22,Christina Vote,2021-22


In [106]:
dfJersey=dfJersey.sort_values('number').drop_duplicates()

In [110]:
fileName=('JerseyNumbersWomens.txt')
jerseyList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    for i in rows:
        col=i.split('\t')
        jerseyDict={'number':int(col[0]),
                    'player':col[1],
                    'season':col[-1]}
        jerseyList.append(jerseyDict)
dfJerseyWomens=pd.DataFrame(jerseyList)